<a href="https://colab.research.google.com/github/pratikagithub/All-About-Data-Science/blob/main/Hybrid_Machine_Learning_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Hybrid machine learning models combine different types of algorithms to leverage their unique strengths, which results in improved predictive performance and robustness.

**When to Build a Hybrid Machine Learning Model?**

Build a hybrid machine learning model when a single algorithm cannot capture data complexity. Different data types or patterns may require this. For example, use a hybrid approach when handling sequential patterns and broader trends in the data.

Combine models like LSTM for sequence learning and Linear Regression for trend analysis to improve performance.

Identify the need for a hybrid model when single models perform poorly based on performance metrics. Different models combined can provide unique strengths in predictive modeling.

Now, I’ll take you through a step-by-step guide on building a hybrid machine learning model where we will be combining the predictive power of two different models to create a hybrid model.

In [1]:
import pandas as pd
from google.colab import files
uploaded = files.upload()
data = pd.read_csv('/content/apple_stock_data.csv')
print(data.head())

Saving apple_stock_data.csv to apple_stock_data.csv
                        Date   Adj Close       Close        High         Low  \
0  2023-11-02 00:00:00+00:00  176.665985  177.570007  177.779999  175.460007   
1  2023-11-03 00:00:00+00:00  175.750671  176.649994  176.820007  173.350006   
2  2023-11-06 00:00:00+00:00  178.317520  179.229996  179.429993  176.210007   
3  2023-11-07 00:00:00+00:00  180.894333  181.820007  182.440002  178.970001   
4  2023-11-08 00:00:00+00:00  181.958893  182.889999  183.449997  181.589996   

         Open    Volume  
0  175.520004  77334800  
1  174.240005  79763700  
2  176.380005  63841300  
3  179.179993  70530000  
4  182.350006  49340300  


As the dataset is based on stock market data, I’ll convert the date column to a datetime type, set it as the index, and focus on the Close price:

In [2]:
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)
data = data[['Close']]

**Choosing the Hybrid Models**

We will be using LSTM (Long Short-Term Memory) and Linear Regression models for this task. I chose LSTM because it effectively captures sequential dependencies and patterns in time-series data, which makes it suitable for modelling stock price movements influenced by historical trends.


Linear Regression, on the other hand, is a straightforward model that captures simple linear relationships and long-term trends in data. By combining these two models into a hybrid approach, we leverage the LSTM’s ability to model complex time-dependent patterns alongside the Linear Regression’s ability to identify and follow broader trends. This combination aims to create a more balanced and accurate prediction system.

So, let’s scale the Close price data between 0 and 1 using MinMaxScaler to ensure compatibility with the LSTM model:

In [3]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
data['Close'] = scaler.fit_transform(data[['Close']])

Now, let’s prepare the data for LSTM by creating sequences of a defined length (e.g., 60 days) to predict the next day’s price:

In [4]:
import numpy as np
def create_sequences(data, seq_length=60):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)

seq_length = 60
X, y = create_sequences(data['Close'].values, seq_length)

Now, we will split the sequences into training and test sets (e.g., 80% training, 20% testing):

In [5]:
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

Now, we will build a sequential LSTM model with layers to capture the temporal dependencies in the data:

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

lstm_model = Sequential()
lstm_model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
lstm_model.add(LSTM(units=50))
lstm_model.add(Dense(1))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Now, we will compile the model using an appropriate optimizer and loss function, and fit it into the training data:

In [7]:
lstm_model.compile(optimizer='adam', loss='mean_squared_error')
lstm_model.fit(X_train, y_train, epochs=20, batch_size=32)

Epoch 1/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 50ms/step - loss: 0.1766
Epoch 2/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 51ms/step - loss: 0.0303
Epoch 3/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - loss: 0.0259
Epoch 4/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - loss: 0.0184
Epoch 5/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 0.0202
Epoch 6/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - loss: 0.0129
Epoch 7/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 0.0134
Epoch 8/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - loss: 0.0116
Epoch 9/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 0.0109
Epoch 10/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 0.0098
Epoch 11/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - loss: 0.0095
Epoch 12/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - loss: 0.0098
Epoch 13/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 84ms/step - loss: 0.0099
Epoch 14/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step - loss: 0.0091
Epoch 15/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 103ms/step - loss: 0.0093
Epoch 16/20
5/5 ━━━━━━━━━━━━━━━━━

Now, let’s train the second model. I’ll start by generating lagged features for Linear Regression (e.g., using the past 3 days as predictors):

In [8]:
data['Lag_1'] = data['Close'].shift(1)
data['Lag_2'] = data['Close'].shift(2)
data['Lag_3'] = data['Close'].shift(3)
data = data.dropna()

Now, we will split the data accordingly for training and testing:

In [9]:
X_lin = data[['Lag_1', 'Lag_2', 'Lag_3']]
y_lin = data['Close']
X_train_lin, X_test_lin = X_lin[:train_size], X_lin[train_size:]
y_train_lin, y_test_lin = y_lin[:train_size], y_lin[train_size:]

Now, let’s train the linear regression model:

In [10]:
from sklearn.linear_model import LinearRegression
lin_model = LinearRegression()
lin_model.fit(X_train_lin, y_train_lin)

LinearRegression()

Now, here’s how to make predictions using LSTM on the test set and inverse transform the scaled predictions:

In [11]:
X_test_lstm = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))
lstm_predictions = lstm_model.predict(X_test_lstm)
lstm_predictions = scaler.inverse_transform(lstm_predictions)

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 543ms/step


Here’s how to generate predictions using Linear Regression and inverse-transform them:

In [12]:
lin_predictions = lin_model.predict(X_test_lin)
lin_predictions = scaler.inverse_transform(lin_predictions.reshape(-1, 1))

And, here’s how to use a weighted average to create hybrid predictions:

In [14]:
print(f"LSTM Predictions Shape: {lstm_predictions.shape}")
print(f"Linear Predictions Shape: {lin_predictions.shape}")

LSTM Predictions Shape: (39, 1)
Linear Predictions Shape: (96, 1)


In [15]:
# Example: Trimming the larger array
min_len = min(len(lstm_predictions), len(lin_predictions))
lstm_predictions = lstm_predictions[:min_len]
lin_predictions = lin_predictions[:min_len]

In [16]:
lstm_predictions = lstm_predictions.reshape(-1, 1)
lin_predictions = lin_predictions.reshape(-1, 1)

In [17]:
hybrid_predictions = (0.7 * lstm_predictions) + (0.3 * lin_predictions)

**Predicting using the Hybrid Model**

Let’s see how to make predictions for the next 10 days using our hybrid model. Here’s how to predict the Next 10 Days using LSTM:

In [18]:
lstm_future_predictions = []
last_sequence = X[-1].reshape(1, seq_length, 1)
for _ in range(10):
    lstm_pred = lstm_model.predict(last_sequence)[0, 0]
    lstm_future_predictions.append(lstm_pred)
    lstm_pred_reshaped = np.array([[lstm_pred]]).reshape(1, 1, 1)
    last_sequence = np.append(last_sequence[:, 1:, :], lstm_pred_reshaped, axis=1)
lstm_future_predictions = scaler.inverse_transform(np.array(lstm_future_predictions).reshape(-1, 1))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step


Here’s how to predict the Next 10 Days using Linear Regression:

In [19]:
recent_data = data['Close'].values[-3:]
lin_future_predictions = []
for _ in range(10):
    lin_pred = lin_model.predict(recent_data.reshape(1, -1))[0]
    lin_future_predictions.append(lin_pred)
    recent_data = np.append(recent_data[1:], lin_pred)
lin_future_predictions = scaler.inverse_transform(np.array(lin_future_predictions).reshape(-1, 1))

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/

And, here’s how to combine the predictive power of both models to make predictions for the next 10 days:

In [20]:
hybrid_future_predictions = (0.7 * lstm_future_predictions) + (0.3 * lin_future_predictions)

Here’s how to create the final DataFrame to look at the predictions:

In [21]:
future_dates = pd.date_range(start=data.index[-1] + pd.Timedelta(days=1), periods=10)
predictions_df = pd.DataFrame({
    'Date': future_dates,
    'LSTM Predictions': lstm_future_predictions.flatten(),
    'Linear Regression Predictions': lin_future_predictions.flatten(),
    'Hybrid Model Predictions': hybrid_future_predictions.flatten()
})
print(predictions_df)

                       Date  LSTM Predictions  Linear Regression Predictions  \
0 2024-11-02 00:00:00+00:00        230.164337                     230.355192   
1 2024-11-03 00:00:00+00:00        229.709167                     225.707291   
2 2024-11-04 00:00:00+00:00        229.257568                     222.703426   
3 2024-11-05 00:00:00+00:00        228.810471                     230.631535   
4 2024-11-06 00:00:00+00:00        228.367401                     225.486380   
5 2024-11-07 00:00:00+00:00        227.927353                     222.494588   
6 2024-11-08 00:00:00+00:00        227.489258                     230.930195   
7 2024-11-09 00:00:00+00:00        227.052429                     225.245599   
8 2024-11-10 00:00:00+00:00        226.616592                     222.284007   
9 2024-11-11 00:00:00+00:00        226.181641                     231.252375   

   Hybrid Model Predictions  
0                230.221594  
1                228.508605  
2                227.291329  